# Analyze Standard Survey Conditions

This notebook analyzes whether BrightSpring has received more conditions in (re-)certification surveys (referred to as **Standard** surveys in QCOR) than other providers in the seven states with the most for-profit facilities.

In [1]:
import pandas as pd
import glob

In [2]:
ownership_data = pd.concat([
    pd.read_csv(f) for f in glob.glob("../../data/ownership/final/*.csv")
])

In [3]:
ownership_data.head()

,provider_id,name,type,region,state,address,phone,address.1,particip_date,certified_beds,hospital_based,ownership_type,termination_code,termination_date,legal_owner,is_brightspring
0,51G002,5TH AVENUE GROUP HOME,Intermediate Care Facilities for Individuals w...,(III) Philadelphia,WV,"916 FIFTH AVENUE\nSAINT ALBANS, WV 25177",304 720-0015,"916 FIFTH AVENUE\nSAINT ALBANS, WV 25177",1981-10-06,9,No,For Profit,NaN,NaN,"RSCR WEST VIRGINIA, INCORPORATED",True
1,51G003,POTOMAC CENTER,Intermediate Care Facilities for Individuals w...,(III) Philadelphia,WV,"ONE BLUE STREET\nROMNEY, WV 26757",304 822-3861,"ONE BLUE STREET\nROMNEY, WV 26757",1983-11-09,24,No,Non-Profit,NaN,NaN,POTOMAC CENTER,False
2,51G007,CROSS LANES GROUP HOME,Intermediate Care Facilities for Individuals w...,(III) Philadelphia,WV,"5202 LINDA VISTA DRIVE\nCROSS LANES, WV 25313",304 776-2005,"5202 LINDA VISTA DRIVE\nCROSS LANES, WV 25313",1986-11-17,8,No,For Profit,NaN,NaN,"RSCR WEST VIRGINIA, INCORPORATED",True
3,51G008,EIGHTH AVENUE GROUP HOME,Intermediate Care Facilities for Individuals w...,(III) Philadelphia,WV,"1519 8TH AVENUE\nHUNTINGTON, WV 25701",304 523-0177,"1519 8TH AVENUE\nHUNTINGTON, WV 25701",1987-03-06,8,No,For Profit,NaN,NaN,"VOCA CORPORATION OF WEST VIRGINIA, INCORPORATED",True
4,51G009,VIRGINIA AVENUE GROUP HOME,Intermediate Care Facilities for Individuals w...,(III) Philadelphia,WV,"821 VIRGINIA AVENUE\nHUNTINGTON, WV 25701",304 523-0196,"821 VIRGINIA AVENUE\nHUNTINGTON, WV 25701",1987-03-06,8,No,For Profit,NaN,NaN,"VOCA CORPORATION OF WEST VIRGINIA, INCORPORATED",True


In [4]:
len(ownership_data)

3946

In [5]:
ownership_data["state"].value_counts()

CA    1166
TX     829
LA     573
OH     487
IN     484
NC     338
WV      69
Name: state, dtype: int64

In [6]:
surveys = pd.read_csv("../../data/qcor/surveys.csv", parse_dates=["date"])
deficiencies = pd.read_csv("../../data/qcor/deficiencies.csv", parse_dates=["date"])

In [7]:
surveys.head()

,survey_key,provider_id,date,type,subtype,num_deficiencies
0,01G005-2010-0,01G005,2010-11-24,STANDARD,LIFE SAFETY,43
1,01G005-2010-1,01G005,2010-11-24,STANDARD,HEALTH,5
2,01G005-2011-0,01G005,2011-07-27,STANDARD,LIFE SAFETY,76
3,01G005-2011-1,01G005,2011-07-27,STANDARD,HEALTH,18
4,01G006-2010-0,01G006,2010-07-27,STANDARD,LIFE SAFETY,0


In [8]:
survey_df = surveys.loc[
    lambda x: 
        x["provider_id"].isin(ownership_data["provider_id"]) & 
        (x["date"] >= "2019-03-05") & # Date KKR bought BrightSpring
        (x["date"] <= "2021-12-31") & # Date data goes to
        (x["type"] == "STANDARD") & # Only Standard surveys
        (x["subtype"] == "HEALTH") # Only Health surveys
].copy()

In [9]:
len(survey_df)

6861

In [10]:
deficiencies.sample(5)

,survey_key,level,tag,desc,date
355665,45G188-2012-0,Standard,K0352,Sprinkler System - Supervisory Signals,2012-04-05
179133,16G055-2016-1,Standard,W0249,PROGRAM IMPLEMENTATION,2016-02-24
64988,07G246-2010-0,Standard,W0218,INDIVIDUAL PROGRAM PLAN,2010-07-15
263561,34G103-2010-1,Condition,W0318,HEALTH CARE SERVICES,2010-01-12
108779,14G262-2011-6,Standard,K0169,Building Construction Type and Height,2011-06-08


In [11]:
con_df = deficiencies.loc[
    lambda x: 
        (x["level"] == "Condition") &
        x["survey_key"].isin(survey_df["survey_key"])
]

In [12]:
len(con_df)

686

In [13]:
con_surveys = (
    con_df
    .groupby("survey_key")["tag"]
    .count()
    .to_frame()
    .rename(columns={"tag": "condition_count"})
    .reset_index()
)

In [14]:
con_surveys.head()

,survey_key,condition_count
0,05G020-2020-3,2
1,05G036-2020-2,1
2,05G090-2020-3,1
3,05G100-2019-3,2
4,05G105-2021-1,1


## Analysis Versus Everyone

In [15]:
overall_results = (
    survey_df
    .merge(
        con_surveys,
        how="left",
        on="survey_key"
    )
    .assign(condition_count = lambda df: df["condition_count"].fillna(0))
    .merge(
        ownership_data[["provider_id", "name", "state", "is_brightspring"]]
    )
    .groupby(["state", "is_brightspring"])
    .pipe(lambda grp: pd.DataFrame({
        "facs": grp["provider_id"].nunique(),
        "surveys": grp["survey_key"].nunique(),
        "conditions": grp["condition_count"].sum()
    }))
    .assign(conditions_per_100_surveys = lambda df: df["conditions"] / df["surveys"] * 100)
)

In [16]:
overall_results.round(1)

facs  surveys  conditions  conditions_per_100_surveys
state is_brightspring                                                       
CA    False             969     1270       124.0                         9.8
      True               88      132        22.0                        16.7
IN    False             377      820       135.0                        16.5
      True               89      190        79.0                        41.6
LA    False             462     1026        42.0                         4.1
      True               71      141        11.0                         7.8
NC    False             296      609        17.0                         2.8
      True               42       82         1.0                         1.2
OH    False             359      494        32.0                         6.5
      True               79      112        13.0                        11.6
TX    False             613     1394        63.0                         4.5
      True              194      460       105.0                        22.8
WV    False              22       43         9.0                        20.9
      True               46       88        33.0                        37.5

The final results below are BrightSpring's total conditions per 100 surveys (**True** column) versus everyone else (**False** column), the ratio of the two values, and the difference of the two values for each state.

In [17]:
(
    overall_results["conditions_per_100_surveys"]
    .unstack()
    .assign(vs_rest_ratio = lambda df: df[True] / df[False])
    .assign(vs_rest_difference = lambda df: df[True] - df[False])
    .sort_values("vs_rest_ratio", ascending=False)
    .round(1)
    [[True, False, "vs_rest_ratio", "vs_rest_difference"]]
)

is_brightspring,True,False,vs_rest_ratio,vs_rest_difference
state,,,,
TX,22.8,4.5,5.1,18.3
IN,41.6,16.5,2.5,25.1
LA,7.8,4.1,1.9,3.7
OH,11.6,6.5,1.8,5.1
WV,37.5,20.9,1.8,16.6
CA,16.7,9.8,1.7,6.9
NC,1.2,2.8,0.4,-1.6


---

---

---